In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Installing Necessary Libraries

!pip install ultralytics
!pip install opencv-python-headless
!pip install pandas

In [3]:
from ultralytics import YOLO
import numpy as np
import cv2
import pandas as pd
import os

# Loading the YOLOv8 model
model = YOLO('yolov8n-pose.pt')  # pretrained YOLOv8n model

In [4]:
# Defining the source video file path
source = '/content/drive/MyDrive/UWO/Term 2/ CS9542 Artificial Intelligence II/Project/480P_2mins/P1.mp4'

In [5]:
# Function to calculate angle between three points

def calculate_angle(p1, p2, p3):
    vector1 = p2 - p1
    vector2 = p3 - p1
    dot_product = np.dot(vector1, vector2)
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)
    angle_radians = np.arccos(dot_product / (magnitude1 * magnitude2))
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

In [7]:
# Processing video frames

cap = cv2.VideoCapture(source)
if not cap.isOpened():
    print("Error: Could not open the video.")
    exit()

all_key = []
frameAngles = []
first_frame = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if first_frame is None:
        first_frame = frame.copy()

    results = model.predict(frame, show=False, conf=0.3, save=False)
    for result in results:
        keypoints = result.keypoints.xy[0]
        if len(keypoints) >= 12:  # Ensure there are enough keypoints
            keys = [(int(keypoint[0]), int(keypoint[1])) for keypoint in keypoints]
            all_key.append(keys)

            # Calculate angle between keypoints 5, 7, and 11
            x1, y1 = keys[5]
            x2, y2 = keys[7]
            x3, y3 = keys[11]

            dot1 = np.array([x1, y1])
            dot2 = np.array([x2, y2])
            dot3 = np.array([x3, y3])

            angle_degrees = calculate_angle(dot1, dot2, dot3)
            frameAngles.append(angle_degrees)

cap.release()

Streaming output truncated to the last 5000 lines.
0: 384x640 1 person, 158.8ms
Speed: 4.2ms preprocess, 158.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 179.3ms
Speed: 3.4ms preprocess, 179.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 163.9ms
Speed: 5.1ms preprocess, 163.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 164.3ms
Speed: 4.9ms preprocess, 164.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 171.0ms
Speed: 5.5ms preprocess, 171.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 184.3ms
Speed: 5.9ms preprocess, 184.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 186.5ms
Speed: 6.6ms preprocess, 186.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 168.1ms
Speed: 5.7ms preprocess,

In [10]:
# Defineing the directory and file path
directory = '/content/drive/MyDrive/UWO/Term 2/CS9542 Artificial Intelligence II/Project/YOLOV8'
file_path = os.path.join(directory, 'P1.xlsx')

# Creating the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Save angles to Excel
df = pd.DataFrame(frameAngles, columns=['Angle (degrees)'])
df.to_excel(file_path, index=False)
print("Angles saved to Excel file.")

Angles saved to Excel file.


In [11]:
# Ensurng the first frame is not empty
if first_frame is not None and first_frame.size > 0 and all_key:
    x1, y1 = all_key[0][5][0], all_key[0][5][1]
    x2, y2 = all_key[0][7][0], all_key[0][7][1]
    x3, y3 = all_key[0][11][0], all_key[0][11][1]

    dot1 = np.array([x1, y1])
    dot2 = np.array([x2, y2])
    dot3 = np.array([x3, y3])

    # Calculating angle
    angle_degrees = calculate_angle(dot1, dot2, dot3)
    print(f"Calculated Angle: {angle_degrees} degrees")

    # Drawing circles and lines on the frame
    circle_radius = 5
    circle_color = (0, 0, 255)  # Red color
    line_thickness = 2
    line_color = (255, 0, 0)  # Blue color

    # Drawing circles
    for dot in [dot1, dot2, dot3]:
        cv2.circle(first_frame, (dot[0], dot[1]), circle_radius, circle_color, -1)

    # Drawing lines
    cv2.line(first_frame, (dot1[0], dot1[1]), (dot2[0], dot2[1]), line_color, line_thickness)
    cv2.line(first_frame, (dot1[0], dot1[1]), (dot3[0], dot3[1]), line_color, line_thickness)

    # Save the image
    cv2.imwrite('/content/drive/MyDrive/UWO/Term 2/CS9542 Artificial Intelligence II/Project/YOLOV8/annotated_P1.jpg', first_frame)
    print("Annotated image saved to Google Drive.")
else:
    print("Frame is empty or no keypoints found.")

Calculated Angle: 40.200815026294066 degrees
Annotated image saved to Google Drive.
